# Compiling a QGL2 AllXY and plotting the output

## Imports

In [ ]:
from pyqgl2.main import compile_function, qgl2_compile_to_hardware
from pyqgl2.test_cl import create_default_channelLibrary
from pyqgl2.qreg import QRegister
from QGL import plot_pulse_files, ChannelLibrary

### Should sequences be compiled to hardware, or just to QGL?

In [ ]:
toHW = True

### Create a test ChannelLibrary; alternatively, load a library you already defined

In [ ]:
create_default_channelLibrary(toHW, True)

# Or create a new ChannelLibrary with channels
# cl = ChannelLibrary(db_resource_name=":memory:")
# q1 = cl.new_qubit('q1')

# Most calls required label and address
# aps2_1 = cl.new_APS2("BBNAPS1", address="192.168.5.101") 
# aps2_2 = cl.new_APS2("BBNAPS2", address="192.168.5.102")
# dig_1  = cl.new_X6("X6_1", address=0)

# Label, instrument type, address, and an additional config parameter
# h1 = cl.new_source("Holz1", "HolzworthHS9000", "HS9004A-009-1", power=-30)
# h2 = cl.new_source("Holz2", "HolzworthHS9000", "HS9004A-009-2", power=-30)

# Qubit q1 is controlled by AWG aps2_1, and uses microwave source h1
# cl.set_control(q1, aps2_1, generator=h1)

# Qubit q1 is measured by AWG aps2_2 and digitizer dig_1, and uses microwave source h2
# cl.set_measure(q1, aps2_2, dig_1.ch(1), generator=h2)

# The AWG aps2_1 is the master AWG, and distributes a synchronization trigger on its second marker channel
# cl.set_master(aps2_1, aps2_1.ch("m2"))
# cl.commit()

## Create needed qubit(s)

In [ ]:
# For QGL2, use a QRegister, not a QGL Qubit
q = QRegister(1)

## Compile to QGL1

### To turn on debug output, uncomment the next 4 lines

In [ ]:
#from pyqgl2.ast_util import NodeError
#from pyqgl2.debugmsg import DebugMsg

#DebugMsg.set_level(1)
#NodeError.MUTE_ERR_LEVEL = NodeError.NODE_ERROR_NONE

In [ ]:
# Insert proper path to QGL2 source and name of qgl2main if not so marked
# Here we compile the named function in the named file from QGL2 to QGL1 and return the new function

In [ ]:
qgl1MainFunc = compile_function("../src/python/qgl2/basic_sequences/AllXY.py", "AllXY", (q,))

## Generate pulse sequences

In [ ]:
# Now run the QGL1 function, producing a list of sequences
seqs = qgl1MainFunc()

## Optionally compile to machine instructions

In [ ]:
if toHW:
    from IPython.display import display
    metaFileName = qgl2_compile_to_hardware(seqs, "AllXY/AllXY")
    print(f"Generated sequence details in '{metaFileName}'")
    # Plot the sequences
    p = plot_pulse_files(metaFileName)
    # Explicitly display the graph which fails to auto-draw in some cases
    display(p)
else:
    from QGL.Scheduler import schedule
    from IPython.lib.pretty import pretty
    print(pretty(schedule(seqs)))